# [--- Day 3: Gear Ratios ---](https://adventofcode.com/2023/day/3)

* Themes: 2D Grids

## Setup

Use the `input` file if present, otherwise use the sample input.

In [1]:
from pprint import pprint
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image
import requests

verbose = False
is_sample = False

try:
  input = [r.strip() for r in open("input", "r").readlines()]
except FileNotFoundError:
  input = """467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598.."""
  input = input.split("\n")
  verbose = True
  is_sample = True

if verbose:
  pprint(input)



['467..114..',
 '...*......',
 '..35..633.',
 '......#...',
 '617*......',
 '.....+.58.',
 '..592.....',
 '......755.',
 '...$.*....',
 '.664.598..']


## Approach and useful bits

The approach I took was to collect all the numbers in the grid into a list that can be used later to query if symbols are near them or not.

Class to represent a number and the location of its first character in the grid.

In [2]:
class Number():
  def __init__(self, text, i, j):
    self.text = text
    self.i = i
    self.j = j

  def __repr__(self):
    return f"({self.text} {self.i} {self.j})"

In [3]:
def get_numbers(line):
  """ Return a list of numbers found on a line with its starting index """
  num_on = False
  numbers = []
  curr_num = ["", 0]
  for i, c in enumerate(line):
    if c.isdigit():
      if num_on:
        curr_num[0] += c
      else:
        # found a new number
        curr_num[0] += c
        curr_num[1] = i
        num_on = True
    else:
      if num_on:
        num_on = False
        numbers.append(curr_num)
        curr_num = ["", 0]
  if curr_num[0] != "":
    numbers.append(curr_num)
  return numbers

# Test cases
print(get_numbers("..*5..5..23"))

assert(
  get_numbers("..*5..5..23") == [['5', 3], ['5', 6], ['23', 9]]
)


[['5', 3], ['5', 6], ['23', 9]]


Fetch all the numbers from the grid and store them in a global `number_list` so they can be used in the solution.

In [4]:
number_list = []
for i, line in enumerate(input):
  for num in get_numbers(line):
    number_list.append(Number(num[0], i, num[1]))
if verbose:
  print(number_list)

[(467 0 0), (114 0 5), (35 2 2), (633 2 6), (617 4 0), (58 5 7), (592 6 2), (755 7 6), (664 9 1), (598 9 5)]


In [5]:
def next_to_symbol(input, i, j):
  """ Returns `True` if (i, j) is next to a symbol """
  for id in range(i-1, i+2):
    for jd in range(j-1, j+2):
      if id == i and jd == j:
        continue
      if id >= 0 and id < len(input) and jd >= 0 and jd < len(input[0]):
        if input[id][jd] != '.' and not input[id][jd].isdigit():
          return True
  return False

def number_next_to_symbol(input, number):
  """ Returns `True` if number is next to a symbol """
  for j in range(number.j, number.j + len(number.text)):
    if next_to_symbol(input, number.i, j):
      return True
  return False

test_input = [
  ['#','.','.','.'],
  ['.','.','.','.'],
  ['.','.','.','#'],
]

assert(next_to_symbol(test_input, 1, 1) is True)
assert(next_to_symbol(test_input, 0, 2) is False)
assert(next_to_symbol(test_input, 1, 2) is True)


In [6]:
def get_neighboring_numbers(i, j, num_list):
  """ Returns a list of all numbers from `num_list` next to the point (i, j) """
  res = []
  for num in num_list:
    if (
        i >= num.i - 1
        and i <= num.i + 1
        and j >= num.j - 1
        and j <= num.j + len(num.text)
    ):
      if verbose:
        print(f"Found a number: {i} {j} {num}")
      res.append(num)
  return res

In [7]:
def gear_ratio(nums):
  """ Return the gear ratio of a set of gears """
  res = 1
  for num in nums:
    res *= int(num.text)
  return res

## Part 1


In [8]:
total = 0

for number in number_list:
  if number_next_to_symbol(input, number):
    total += int(number.text)

print(total)

if is_sample:
  assert(total == 4361)
else:
  assert(total == 549908)

4361


## Part 2
* Search for all "*" characters
* Fetch numbers around it
* If there's 2 numbers, get the gear ratio and add it to running total

In [9]:
total = 0

for i, line in enumerate(input):
  for j, c in enumerate(line):
    if c == "*":
      neighboring_nums = get_neighboring_numbers(i, j, number_list)
      if len(neighboring_nums) == 2:
        total += gear_ratio(neighboring_nums)
print(total)

if is_sample:
  assert(total == 467835)
else:
  assert(total == 81166799)

Found a number: 1 3 (467 0 0)
Found a number: 1 3 (35 2 2)
Found a number: 4 3 (617 4 0)
Found a number: 8 5 (755 7 6)
Found a number: 8 5 (598 9 5)
467835
